# Item 43: Inherit from `collections.abc` for Custom Container Types

In [1]:
# When we're designing classes for simple uses cases like sequences, its natural to want to subclass
# Python's built-in list type directly
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)

    def frequency(self):
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0) + 1
        return counts

# With the code above, we've essentially extended the functionality of the built-in list container
foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
print('Length is', len(foo))
foo.pop()
print('After pop:', repr(foo))
print('Frequency:', foo.frequency())

Length is 7
After pop: ['a', 'b', 'a', 'c', 'b', 'a']
Frequency: {'a': 3, 'b': 2, 'c': 1}


In [2]:
# Now imagine we want to provide an object that feels like a list and allows indexing but isn't a list
# subclass. For example, say we want to provide sequence semantics (like list or tuple) for a binary tree class
class BinaryNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

In [3]:
# How can we make this class act like a sequence type? 
bar = [1, 2, 3]
bar[0]
# Python interprets the above as
bar.__getitem__(0)

1

In [4]:
# To make the BinaryNode class act like a sequency, we can provide a custom implementation of __getitem__
# that traverses the object depth-first
class IndexableNode(BinaryNode):
    def _traverse(self):
        if self.left is not None:
            yield from self.left._traverse()
        yield self
        if self.right is not None:
            yield from self.right._traverse()

    def __getitem__(self, index):
        for i, item in enumerate(self._traverse()):
            if i == index:
                return item.value
        raise IndexError(f'Index {index} is out of range')


In [5]:
# We can construct our binary tree as usual
tree = IndexableNode(
    10,
    left=IndexableNode(
        5,
        left=IndexableNode(2),
        right=IndexableNode(
            6, 
            right=IndexableNode(7))),
    right=IndexableNode(
        15,
        left=IndexableNode(11)))

In [6]:
# But we can also access uit like a list in addition to being able to traverse the tree with the left and
# right attributes
print('LRR is', tree.left.right.right.value)
print('Index 0 is', tree[0])
print('Index 1 is', tree[1])
print('11 in the tree?', 11 in tree)
print('17 in the tree?', 17 in tree)
print('Tree is', list(tree))

LRR is 7
Index 0 is 2
Index 1 is 5
11 in the tree? True
17 in the tree? False
Tree is [2, 5, 6, 7, 10, 11, 15]


In [7]:
# The problem here is that implementing __getitem__ is not enough to provide all of the sequence semantics
# you'd expect from a list instance
len(tree)

TypeError: object of type 'IndexableNode' has no len()

In [9]:
# The len built-in function requires another special method named __len__ to be implemented for a custom
# sequence type
class SequenceNode(IndexableNode):
    def __len__(self):
        for count, _ in enumerate(self._traverse(), 1):
            pass
        return count


tree = SequenceNode(
    10,
    left=SequenceNode(
        5,
        left=SequenceNode(2),
        right=SequenceNode(
            6,
            right=SequenceNode(7))),
    right=SequenceNode(
        15,
        left=SequenceNode(11)))

print('Tree length is', len(tree))


Tree length is 7


This is, however, still not enought for the class to be a valid sequence (`count` and `index` methods are missing). To avoid this hurtle of creating container types by yoursell from scratch, we can use the built-in `collections.abc` module which defines a set of abstract base classes that provide all the typical methods for each container type.  

In [10]:
# When we subclass from these abstract base classes and forget to implement required methods, the module
# tells us something is wrong
from collections.abc import Sequence

class BadType(Sequence):
    pass

foo = BadType()

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

In [11]:
# When we do implement all methods required by an abstract base class from collections.abc, it provides all
# of the traditional methods, like index and count for free
class BetterNode(SequenceNode, Sequence):
    pass


tree = BetterNode(
    10,
    left=BetterNode(
        5,
        left=BetterNode(2),
        right=BetterNode(
            6,
            right=BetterNode(7))),
    right=BetterNode(
        15,
        left=BetterNode(11)))

print('Index of 7 is', tree.index(7))
print('Count of 10 is', tree.count(10))

Index of 7 is 3
Count of 10 is 1
